# TODO
1. Import DENA and BETD to python
2. Import date, document, and parent organization to df
3. Translate to english
4. Find out which type of regression to use
5. Reduce text complexity processes
6. Perform topic modeling
7. Perform similarity comparison of topic models
8. Add similarity to df and perform regression

# Topic Modeling of Solar Renewable Energy Projects

# 1) Data

## 1a) Create main dataframe

## 1b) Add DENA info to df

In [6]:
import os # reads directory of stored DENA pdfs
from PyPDF2 import PdfReader # reads pdfs
import re # regex is used for finding date published of DENA report
import pandas as pd

directory  = 'DENA'


df = pd.DataFrame(columns = ["name", "year", "doc", "BETD", "DENA", "news", "txt_sim_news", "txt_sim_fce", "txt_sim_proj"])

for filename in os.listdir(directory): # for loop for importing all DENA files to a dataframe
    f = os.path.join(directory,filename)
    if os.path.isfile(f):
        reader = PdfReader(f)
        n_pages = reader._get_num_pages()
        n_pages = n_pages-1
        n_text = []
        document = ''
        # TODO: FIX LOOP FOR PAGES
        for i in int(n_pages):
            page = reader.pages[n_pages]
            text = page.extract_text()
            document = document + text

        reg_date = re.findall('2017|2018|2019|2020|2021|2022', document)

        year = ''
        if reg_date:
            year  = reg_date[0]
        if not reg_date:
            reg_date = re.findall('2017|2018|2019|2020|2021|2022', filename)
            if reg_date:
                year = reg_date[0]
            if not reg_date:
                year = 'na'

       # insert to df
        df.loc[-1] = [filename, year ,document, 0, 1, 0, 'na', 'na', 'na']
        df.index = df.index + 1
        df = df.sort_index()
    # print(filename.rstrip('.pdf'))
print(df)
pd.set_option('display.max_colwidth', None)
print(df['doc'].iloc[2])

TypeError: 'int' object is not iterable

In [ ]:
# TEAM: make this cell to loop through df and manually add year at index, n=65

In [ ]:
# for loop for importing all BETD files to a dataframe

## 1c) Detect language in df cell and translate inplace

In [ ]:
# DeepL api integration (must have account for api to run cell)